# Pima indianas Diabetes Database

Hello, today i am going to  estimate the liklihood of person to be diabetic, this is my first MachineLearning real application away from the theory.
 


## Prepare , load the data 

first things first, import the modules needed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
filename = '../input/diabetes.csv'
data=pd.read_csv(filename)

let's take a very quick look at our data

In [ ]:
print(data.columns) # to know all the features(variables) we got in our data

In [ ]:
print(data.head()) #the first 5 rows

 ## First things first: analysing the outcome
 
 by the outcome we mean whether the person is diabetic or no (1 = yes; 0 = no)

descriptive statistics summary :

 
 

In [ ]:
data['Outcome'].describe()


now the histogram of the outcome : 


In [ ]:
data['Outcome'].hist(figsize=(7,7))

we can easily see that  the Non-Diabetic persons is more than the diabetics by what it seems the half


#  ' Diabetes ' feature Relationships !!!!

**Diabetes** love Aged,ppl with high level of Glucose  " based from what I learned from high School", 
So let's start with them, by that i mean there relation with Diabetes.



In [ ]:

#borrowed from my friend Ayoub Benaissa.
def plot_diabetic_per_feature(data, feature):
    grouped_by_Outcome = data[feature].groupby(data["Outcome"])
    diabetic_per_feature = pd.DataFrame({"Sick": grouped_by_Outcome.get_group(1),
                                        "Not Sick": grouped_by_Outcome.get_group(0),
                                        })
    hist = diabetic_per_feature.plot.hist(bins=60, alpha=0.6)
    hist.set_xlabel(feature)
    plt.show()
    



let's start with the age :

In [ ]:
plot_diabetic_per_feature(data, "Age")


*Intersting*, as u see, The older the persons,The higher the number of diabetics.
ofc that's based from the dataset ' my doctor friend told me there's 2 types of Diabetes, so maybe this data concern the one who target the olders"


In [ ]:
plot_diabetic_per_feature(data, "Glucose")


Same ... the more Glucose you have to more likley to have Diabetes, i guess that will be pretty obvious for you  if you know bit of biology.
We notice the odd 0 information, let's confirm: 



In [ ]:
print(data["Glucose"].min())

that true, I don't think that a person can have 0 Glucose and still alive, so i'll delete it later + It also may effect the accuracy of the model.


What about the **BMI** ( it's basically the weight of the person with kg) :


In [ ]:
plot_diabetic_per_feature(data, "BMI")


very reasonable results, same notes from the 2 above, we can also notice the 0 odd info so we'll delete it late for the same reasons.

# Think beyond the box
let's think beyond the box, how is that? 
    with :
* Correlation matrix
* Scatter plots between the most correlated variables


**Correlation matrix  : **


In [ ]:
import seaborn as sns #the librery we'll use for the job xD

corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, cbar=True, annot=True, square=True, vmax=.8);

Alright, let's break this matrix down to some few notes : 
* Glucose, Age and  BMI are the most Correlated features with the 'Outcome'
* Bloodpressure, SkinThikness have tiny Correlation with the outcome, hummm !
* check how the SkinThikness and BMI Correlated, make me think of rolling it out since mose of the fat ppl tends to have thicc skin
* Age with Pregnancies are the most Correlated features
* Insulin with Glucuse ' BIOLOGY  :) "
* DiabetesPedigreeFunction bit Correlated with most of them ' I am not sure with feature really mean"
* finnaly SkinThikness with Insulin, that's odd !

### Scatter plots between 'Outcome' and correlated variables

In [ ]:
sns.set()
cols = ['Pregnancies','Glucose','BloodPressure','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']
sns.pairplot(data[cols], size = 2.5)
plt.show();

opss! that's a huge amount of graphs to analyse but let's carry on :
* bloodpressure and age tend to have a relation and that's kinda obviose since most of aged ppl have bloodpressure
* Glucose and insulin have very strong relation makes me think about deleting the inslin futuere
* in pregnancy/age we notice some kind of a liniar line in the right bottom 

that's it, no need to repeat the past notes ' even tho i did '



# Outliers 
Outliers is also something that we should be aware of. Why? Because outliers can markedly affect our models.

let's see what we can do :

In [ ]:
data.min()


It's okay to have 0 Pregnancies, for the rest, we'll delete the rows containing the 0 values:


In [ ]:
data = data.drop(data[data['Glucose'] == 0].index)
data = data.drop(data[data['SkinThickness'] == 0].index) # even it will be deleted xD
data = data.drop(data[data['BloodPressure'] == 0].index) #same
data = data.drop(data[data['BMI'] == 0].index)
data = data.drop(data[data['Insulin'] == 0].index)

print(data.min()) # let's check




*excellent*, NEXT !!

# Preparing the data/Selecting a model



In [ ]:
#just the libreries we need
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts


#preparing the data
Y=data['Outcome']
cols = ['Pregnancies','Glucose','DiabetesPedigreeFunction','Insulin','BMI','Age']
# I deleted BloodPressure and Skinthikness

X_train, X_test, Y_train, Y_test = tts(data[cols], Y, test_size = 0.2, random_state = 2)



#Selecting a model

lr=LogisticRegression()
lr.fit(X_train,Y_train)
accuracy=lr.score(X_test,Y_test)
print("LogisticRegression Accuracy: {}%".format(accuracy * 100))


Gs = GaussianNB()
Gs.fit(X_train,Y_train)
accuracy=Gs.score(X_test,Y_test)
print("GaussianNB Accuracy: {}%".format(accuracy * 100))

Kn = KNeighborsClassifier(n_neighbors=12)  
Kn.fit(X_train, Y_train)
accuracy=Kn.score(X_test,Y_test)
print("KNeighborsClassifier Accuracy: {}%".format(accuracy * 100))


Ds = DecisionTreeClassifier()  
Ds.fit(X_train, Y_train) 
accuracy=Ds.score(X_test,Y_test)
print("DecisionTreeClassifier Accuracy: {}%".format(accuracy * 100))

Rs = RandomForestClassifier(n_estimators=63, random_state=0)  
Rs.fit(X_train, Y_train)  
accuracy=Rs.score(X_test,Y_test)
print("RandomForestClassifier Accuracy: {}%".format(accuracy * 100))

I'll go with RandomForestClassifier for having the highest Accuracy ( didn't study the variance in this version so wait for the next one )

**thanks for reading  and to the next version/kernels **